In [ ]:
import anytree
from collections import Counter, defaultdict
from copy import copy
from hstrat import hstrat
from interval_search import doubling_search
from iterpop import iterpop as ip
import itertools as it
from keyname import keyname as kn
from nbmetalog import nbmetalog as nbm
import numpy as np
import opytional as opyt
import pandas as pd
from phylotrackpy import systematics
import random
from slugify import slugify
import sys
from tqdm import tqdm
import typing


In [ ]:
np.random.seed(1)
random.seed(1) # ensure reproducibility
sys.setrecursionlimit(1000000)


In [ ]:
nbm.print_metadata()


# Pick Hereditary Stratigraph Configurations


In [ ]:
def make_conditions(num_generations: int) -> pd.DataFrame:
    res = []
    for condemner_factory, target_column_bits, differentia_bit_width in it.product(
        [
            hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution,
            hstrat.StratumRetentionCondemnerRecencyProportionalResolution,
        ],
        [
            1024,
            1024 * 8,
            1024 * 16,
        ],
        [
            1,
            8,
            64,
        ],
    ):
        policy_param = doubling_search(
            lambda x: \
                condemner_factory(x + 1).CalcNumStrataRetainedExact(num_generations)
                * differentia_bit_width > target_column_bits or x >= num_generations,
            1 if condemner_factory == hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution else 0,
        )

        actual_column_strata = condemner_factory(policy_param).CalcNumStrataRetainedExact(num_generations)
        actual_column_bits = actual_column_strata * differentia_bit_width

        res.append({
            'Retention Policy' : condemner_factory.__name__[25:],
            'Differentia Bit Width' : differentia_bit_width,
            'Retention Policy Resolution Parameter' : policy_param,
            'Target Retained Bits' : target_column_bits,
            'Actual Retained Bits' : actual_column_bits,
            'Retained Bits Error' : actual_column_bits - target_column_bits,
            'Actual Retained Strata' : actual_column_strata,
            'condemner' : condemner_factory(policy_param),
        })
    return pd.DataFrame.from_records(res)


In [ ]:
# picked in another data frame to minimize retained bits error
# for fair comparisons between configurations
conditions_df = make_conditions(36864)
conditions_df.drop('condemner', axis=1)


In [ ]:
make_bundle = lambda: hstrat.HereditaryStratigraphicColumnBundle({
    kn.pack({
        'differentia' : row['Differentia Bit Width'],
        'policy' : row['Retention Policy'],
        'resolution' : row['Retention Policy Resolution Parameter'],
        'target_bits' : row['Target Retained Bits'],
        'actual_bits' : row['Actual Retained Bits'],
        'bits_error' : row['Retained Bits Error'],
        'actual_strata' : row['Actual Retained Strata'],
    }) \
        : hstrat.HereditaryStratigraphicColumn(
            stratum_differentia_bit_width=row['Differentia Bit Width'],
            stratum_retention_condemner=row['condemner'],
            stratum_ordered_store_factory=hstrat.HereditaryStratumOrderedStoreDict,
    )
    for __, row in conditions_df.iterrows()
})


# Set Up Population


In [ ]:
uid_generator = it.count()


In [ ]:
tracker = systematics.Systematics(lambda org: org.uid, True, True, False, False)


In [ ]:
class Organism:
    def __init__(self, genotype):
        self.uid = next(uid_generator)
        self.genotype = genotype
        self.hstrat_column = make_bundle()
#         self.taxon = systematics.Taxon(self)

    def CloneDescendant(self):
        offspring = copy(self)
        offspring.hstrat_column = self.hstrat_column.CloneDescendant()
        offspring.uid = next(uid_generator)
        offspring.genotype += np.random.normal()
#         offspring.taxon = tracker.add_org(offspring, self.taxon)
        return offspring


In [ ]:
population = [
    Organism(phenotype)
    for phenotype in range(100)
]


# Do Evolution


In [ ]:
def do_generation(population: typing.List[Organism]) -> typing.List[Organism]:

    def get_fitness(individual: Organism) -> float:
           return individual.genotype

    def do_tournament(tournament_size: int) -> Organism:
        contestants = random.sample(population, tournament_size)
        winner = max(contestants, key=get_fitness)
        return winner.CloneDescendant()

    new_population = [
        do_tournament(7) for __ in population
    ]
    return new_population


In [ ]:
# for generation in tqdm(range(36864 - 1)):
for generation in tqdm(range(100)):
    population = do_generation(population)


# Extract Pairwise MRCA Estimates for Extant Organisms


In [ ]:
res = []
for extant1, extant2 in tqdm([*it.product(population, population)]):
    if extant1 != extant2:
        bounds = extant1.hstrat_column.CalcRankOfMrcaBoundsWith(extant2.hstrat_column)
        for impl in extant1.hstrat_column:
            res.append({
                'Genotype' \
                    : extant1.genotype,
                'Column Configuration' \
                    : impl,
                'Differentia Bit Width' \
                    : kn.unpack(impl)['differentia'],
                'Stratum Retention Policy' \
                    : kn.unpack(impl)['policy'],
                'Stratum Retention Policy Resolution Parameter' \
                    : kn.unpack(impl)['resolution'],
                'Stratigraphic Column Actual Retained Bits' \
                    : extant1.hstrat_column[impl].GetNumStrataRetained() * int(kn.unpack(impl)['differentia']),
                'Stratigraphic Column Target Retained Bits' \
                    : kn.unpack(impl)['target_bits'],
                'Stratigraphic Column Actual Num Retained Strata' \
                    : extant1.hstrat_column[impl].GetNumStrataRetained(),
                'Taxon Compared From' \
                    : extant1.uid,
                'Taxon Compared To' \
                    : extant2.uid,
                'Generation of Taxon Compared From' \
                    : extant1.hstrat_column.GetNumStrataDeposited(),
                'Generation of Taxon Compared To' \
                    : extant2.hstrat_column.GetNumStrataDeposited(),
                'Generation Of MRCA Lower Bound (inclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[0],
                    ),
                'Generation Of MRCA Upper Bound (exclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[0],
                    ),
                'MRCA Bound Confidence' \
                    : extant1.hstrat_column[impl].CalcRankOfMrcaBoundsWithProvidedConfidenceLevel(),
                'Rank of Earliest Detectable Mrca With' \
                    : extant1.hstrat_column[impl].CalcRankOfEarliestDetectableMrcaWith(extant2.hstrat_column[impl]),
            })

res_df = pd.DataFrame.from_records(res)


In [ ]:
res_df


# Save Pairwise MRCA Estimates to File


In [ ]:
res_df.to_csv(
    kn.pack({
        'a' : 'pairwise_mrca_estimates',
        'simulation' : 'evolution_synchronous',
        'tournament_size' : 7,
        'ext' : '.csv.gz',
    }),
    compression='gzip',
)
